# Custom TF-IDF implementation


In this notebook, we endeavor to replicate the TFIDF vectorizer functionality found in Scikit-learn's sklearn library. This replication process is divided into two distinct tasks, namely Task 1 and Task 2.

In Task 1, our objective is to faithfully reproduce the behavior of the fit and transform functions provided by Scikit-learn. Subsequently, we will compare the outcomes of our implementation with those obtained using Scikit-learn's functionality.

Moving on to Task 2, our focus shifts towards extracting the top 50 features. Once again, we will meticulously compare our results to those generated by Scikit-learn's library.

For more comprehensive details and precise definitions, please consult the readme document located in the current folder.

## Task-1

In [29]:
# Collection of string documents - generating a collection of string documents
# Taken from sklearn.feature_extraction.text.TfidfVectorizer exampls
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

### SkLearn Implementation

In [30]:
# Importing TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [31]:
# sklearn feature names, they are sorted in alphabetic order by default.
print(vectorizer.get_feature_names_out())

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']


In [32]:
# Here we will print the sklearn tfidf vectorizer idf values after applying the fit method
# After using the fit function on the corpus the vocab has 9 words in it, and each has its idf value.

print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [33]:
# shape of sklearn tfidf vectorizer output after applying transform method.
skl_output.shape

(4, 9)

In [34]:
# sklearn tfidf values for first line of the above corpus.
# Here the output is a sparse matrix

print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [35]:
# sklearn tfidf values for first line of the above corpus.
# We are converting the sparse output matrix to dense matrix and printing it.
# The output is normalized using L2 normalization. sklearn does this by default.

print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Custom implementation

In [36]:
# importing libraries
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
from scipy.sparse import vstack
import math
import operator
from sklearn.preprocessing import normalize
import numpy

# Adding imports for fit and transform functions
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from collections import Counter

In [37]:
# Writing a set of functions that will cover the task 1: 
# Fit function based on reference document
def fit(dataset):
    '''
    Inputs a list of text sentences and outputs 
    the vocabulary dictionary in alphabetical order
    '''    
    uniquewords = set()
    # Ensuring that the dataset entered is in the list format
    if isinstance(dataset, list):
        for row in dataset: # for each review in the dataset
            for word in row.split(" "): # for each word in the review. Split method converts a string into list of words
                if len(word) < 2:
                    continue
                uniquewords.add(word)
        uniquewords = sorted(list(uniquewords))
        vocab = {j:i for i,j in enumerate(uniquewords)}
        
        return vocab
    else:
        print("Enter the correct format of the data in list")
        
# Transform function based on the reference document
def transform(dataset, vocab):
    '''
    Inputs the dataset and Vocabulary dictionary, both inputed by the user. 
    The vocab inputted is the output of the fit function. 
    This function generates a sparse martrix comprising of the counts of a word 
    occurence for all documents in the corpus. this function is rewritten from the reference document 
    provided
    '''
    
    rowval, colval, valval = [], [], []
    
    # Ensuring the dataset is in the list format
    if isinstance(dataset, list):
        for x, y in enumerate(dataset): # y = documnet, x = index
            doc_word_dict = dict(Counter(y.split(" "))) # Generating dict containing each words as key and number of times of occurence in values
            for a, b in doc_word_dict.items(): # a = word in a doc, b = number of times the word has occured
                if len(a) < 2: # Not considering single words (to remove punctuations and words like a, I etc...)
                    continue # Ensuring the outer for loop continues if the word has < 2 
                # Checking word with vocab dict    
                m = vocab.get(a, -1) # Returns value -1 if the word isn't available
                if m != -1:
                    rowval.append(x) # Taking the value of the index number of the doc
                    colval.append(m) # Taking the value of the word's column number from vocab
                    valval.append(b) # Taking the value of the frequency from the Counter function for a given word in the doc
        
        return csr_matrix((valval, (rowval,colval)), shape = (len(dataset), len(vocab)))
    
    else: 
        return "The given dataset doesn't contain a list of sentances"

def termfreq(matrix):
    '''
    Provides a matrix where each element is 
    the term frequency value of the of inputed 
    matrix
    ''' 
    return matrix/matrix.sum(axis = 1)

def idf_(matrix):
    '''
    Provides a matrix where each element is the Inverse 
    document frequency of the inputted matrix
    The Scikit learn formulae for IDF can be interpretted as: 
    1+log𝑒(1 + Total number of documents in collection1) - log(1+Number of documents with term t in it)
    = 1+ log1p(Total number of documents in collection) - logp(1+ number of document with term t in it)
    = 1 + log(1 + total number of document/ number of rows) - log1p(number of document with term t in it)
    = 1 + log(1 + total number of document/ number of rows) - log1p(COLUMN non-zero sum)
    '''
    # get_shape for scipy.csr_matrix.get_shape provides the shape of the csr matrix, we are trying to get the rowcount or doc count
    # Calculating the log of (1 + total number of documents)
    
    return 1 + math.log(1 + matrix.get_shape()[0]) - np.log1p(matrix.getnnz(axis = 0))

def tfidfvectorzr(tf, idf):
    '''
    multiplies the corresponding idf value with the
    tf value and returns a sparse matrix
    '''
    return csr_matrix(normalize(np.multiply(tf.toarray(),idf)))

In [38]:
# Testing the data with corpus below and comapring with the SkLearn's implementation:

corpus = ['this is the first document',
      'this document is the second document',
      'and this is the third one',
      'is this the first document']

#SkLearn's code for corpus (copied from assignment above)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorz = TfidfVectorizer()
vectorz.fit(corpus)
skl_output = vectorz.transform(corpus)

# checking if the vocabulary from fit function matches with SkLearn's get_featurenames()
vocabulary = fit(corpus)
print("Feature names generated by user generated fit function:")
print(vocabulary)
print("-"*50)
print("Feature name generated by SkLearn:")
print(vectorz.get_feature_names_out())

# checking if idf values derived from  matches with SkLearn's output:
print("="*50)
y = transform(corpus, vocabulary)
w = termfreq(y)
z = idf_(y)

print("\nUser\'s idf_ function generated idf values")
print()
print(z)
print("-"*50)
print("\nSkLearn\'s Vectorizer generated idf values:")
print(vectorz.idf_)

# Testing the tfidfvectorizer formulae:
# Calculation of final TFIDF
print("="*50)
print("\nTFIDF final vector value calculated")
print(tfidfvectorzr(w,z))

#Calculation of final TFIDF from SciKit learn
print("-"*50)
print("\nTFIDF as calculated from the SciKit learn package")
print(vectorz.fit_transform(corpus))


Feature names generated by user generated fit function:
{'and': 0, 'document': 1, 'first': 2, 'is': 3, 'one': 4, 'second': 5, 'the': 6, 'third': 7, 'this': 8}
--------------------------------------------------
Feature name generated by SkLearn:
['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']

User's idf_ function generated idf values

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]
--------------------------------------------------

SkLearn's Vectorizer generated idf values:
[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]

TFIDF final vector value calculated
  (0, 1)	0.4697913855799205
  (0, 2)	0.5802858236844359
  (0, 3)	0.38408524091481494
  (0, 6)	0.38408524091481494
  (0, 8)	0.38408524091481494
  (1, 1)	0.6876235979836938
  (1, 3)	0.281088674033753
  (1, 5)	0.5386476208856762
  (1, 6)	0.281088674033753
  (1, 8)	0.281088674033753
  (2, 0)	0.511848512707169


#### Result for Task 1:

The TFIDF vectors generated by both Scikit-learn and our custom code have produced exactly the same values, confirming the success of our custom implementation.

## Task-2

In [39]:
#local directories:
local = '/Users/rathish/Documents/Data_Science_Projects/Custom_TFIDF_Implementation/data'
%cd $local


/Users/rathish/Documents/Data_Science_Projects/Custom_TFIDF_Implementation/data


In [40]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type
import pickle


with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)

# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [41]:
# Writing a set of modified functions taken from task 1 to complete task 2: 
# The fit, transform and termfreq functions do not change 

# Creating a tf-idf function which generates tf and idf valued matrix using the previously defined fit, transpform and termfreq:
def tf_idf(corpus, top_n_featnames):
    '''
    This funtion generates both tf and idf directly from the input of corpus and top_n_feat names. The dictionary with topfeatures is returned as a third output
     This funtion uses fit, transform, termfreq funcion and idf function
     '''
    # Repeating the first four steps conducted in task 1 to obtain a full idf for the data (so far top_n_featnames haven't been considered)
    vocab = fit(corpus)
    transform_full = transform(corpus, vocab)
    tf = termfreq(transform_full)
    fullidfvector = idf_(tf)
    # Associating the calculated idf values to vocab generated by fit function, output: dict(idf:word)
    fullidfdict = dict([(k,v) for k,v in zip(list(vocab.keys()),fullidfvector.tolist())])
    # Reordering and creating vocab with top 50 features. All other feature values to be marked to zero
    sortedtuplist = sorted(fullidfdict.items(), key=lambda x: x[1], reverse = True)
    
    # Creating two lists to append vocab words and idf values
    m = []
    n = []
    for x in sortedtuplist:
        m.append(x[0])
        n.append(x[1])
    # Making the idf values for vocab which are not in the top_n_featnames zero
    for y in range(top_n_featnames, len(m)):
        n[y] = 0
    
    topfeatdict = dict(zip(m,n))
    
    # Creating another dict that contains only the top_n_features
    rettopfeatdict = topfeatdict.copy()
    for feat in topfeatdict.keys():
        if topfeatdict[feat] == 0:
            del rettopfeatdict[feat]
   
    collist, idflist = [], []
    # Reconstructing new TF and IDF matrices 
    for feat in sorted(topfeatdict.keys()):
        if topfeatdict[feat] != 0:
            colnum = vocab[feat]
            csrmat = tf.toarray()[:,colnum]
            collist.append(csrmat.reshape(-1,1)) # Getting col using the col num and appending it to collist(list)
            
    tftopfeat = np.hstack(collist) # Regular matrix with top feature names
    
    # Generating a row matrix with top idf values
    
    for idf in sorted(topfeatdict.keys()):
        if topfeatdict[idf] != 0:
            idflist.append(topfeatdict[idf])
    
    idf_topfeat = np.array(idflist)
   
    
    return tftopfeat, idf_topfeat, rettopfeatdict
    
    
def tfidfvectorzr2(tf, idf):
    '''
    multiplies the corresponding idf value with the
    tf value and returns a sparse matrix
    ''' 
    # Generating a TF - CSR matrix with only top feature names
    return csr_matrix(normalize(np.multiply(tf,idf)))
    

In [43]:
# Task 2: Testing the data with corpus below and comapring with the SkLearn's implementation:

#importing corpus data
import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)

#SkLearn's code copied from the above cells
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 50)
X = vectorizer.fit_transform(corpus)


a, b, c = tf_idf(corpus, 50)
print(('The shape of custom TFIDF vectorizer = {}'.format(tfidfvectorzr2(a,b).shape)),'\n\n','The shape of sklearn TFIDF Vectorizer ='.format(X.shape))
print(c, '\n',vectorizer.get_feature_names_out() )
print('='*50, '\nCustom TFIDF vector value')
print(tfidfvectorzr2(a,b))
print('\n\n Sklearn TFIDF vector value: ')
print(X)

The shape of custom TFIDF vectorizer = (746, 50) 

 The shape of sklearn TFIDF Vectorizer =
{'aailiyah': 6.922918004572872, 'abandoned': 6.922918004572872, 'abroad': 6.922918004572872, 'abstruse': 6.922918004572872, 'academy': 6.922918004572872, 'accents': 6.922918004572872, 'accessible': 6.922918004572872, 'acclaimed': 6.922918004572872, 'accolades': 6.922918004572872, 'accurate': 6.922918004572872, 'accurately': 6.922918004572872, 'achille': 6.922918004572872, 'ackerman': 6.922918004572872, 'actions': 6.922918004572872, 'adams': 6.922918004572872, 'add': 6.922918004572872, 'added': 6.922918004572872, 'admins': 6.922918004572872, 'admiration': 6.922918004572872, 'admitted': 6.922918004572872, 'adrift': 6.922918004572872, 'adventure': 6.922918004572872, 'aesthetically': 6.922918004572872, 'affected': 6.922918004572872, 'affleck': 6.922918004572872, 'afternoon': 6.922918004572872, 'aged': 6.922918004572872, 'ages': 6.922918004572872, 'agree': 6.922918004572872, 'agreed': 6.9229180045728

#### Result for Task 2: 

We do not obtain the same vectors because, upon closer examination, it becomes evident that Scikit-learn's max_features selects features based on term frequency, whereas the features obtained through our custom implementation are ranked according to the IDF order. In the near future, we plan to write additional code to ensure that the custom feature selection process aligns with term frequency.